# Proyecto análisis de transacciones financieras
## datos tomados de kaggle del siguiente link: https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets?resource=download

Para este proyecto se va a analizar los diferentes registros de transacciones relizados en la década de 2010 para ana´lisis del comportmaiento del cliente y pronósticos de gastos

# FASE 2
### Creación contenedor Docker

docker pull postgres
docker run --name Inteligencia_negocios -e POSTGRES_USER=admin -e POSTGRES_PASSWORD=inteligencia -e POSTGRES_DB=financial -p 5432:5432 -d postgres

## Conexión con la base de datos de postgres

En el proyecto se tiene 3 archivos CSV y 2 archivos jason, se cargará el archivo mas pesado de CSV a la base de datos: "Transactions_data"